In [1]:
! wget http://stat-computing.org/dataexpo/2009/2008.csv.bz2

--2018-07-20 06:57:39--  http://stat-computing.org/dataexpo/2009/2008.csv.bz2
Resolving stat-computing.org (stat-computing.org)... 52.218.212.43
Connecting to stat-computing.org (stat-computing.org)|52.218.212.43|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113753229 (108M) [application/x-bzip2]
Saving to: ‘2008.csv.bz2’

100%[======================================>] 113,753,229 9.33MB/s   in 21s    

2018-07-20 06:58:01 (5.06 MB/s) - ‘2008.csv.bz2’ saved [113753229/113753229]



In [4]:
! tar -xvjf 2008.csv.bz2

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [5]:
! bzip2 -d 2008.csv.bz2

In [6]:
! ls -lrt

total 2019768
-rw------- 1 s7ed-a18f3badb92bc2-a9f6794a31ec users 689413344 Dec  9  2014 2008.csv
drwx------ 2 s7ed-a18f3badb92bc2-a9f6794a31ec users      4096 May 26 02:48 MNIST_data


In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('2008.csv')

In [9]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,...,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [10]:
# For SQL-type queries (Spark)
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import udf

# For regression and other possible ML tools (Spark)
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics

# Important for managing features  (Spark)
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler

# For displaying and other related IPython tools...
from IPython.display import display
from IPython.html.widgets import interact

# Typycal Python tools
import sys
import numpy as np
import pandas as pd
import time
import datetime
import matplotlib.pyplot as plt
import os.path

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [11]:
# To show plots inline
get_ipython().magic(u'matplotlib inline')

In [12]:
textFile = sc.textFile('2008.csv')

In [13]:
textFileRDD = textFile.map(lambda x: x.split(','))
header = textFileRDD.first()

textRDD = textFileRDD.filter(lambda r: r != header)

In [16]:
num_records = textFileRDD.count()
print ('Number of records ' , num_records)

Number of records  7009729


In [17]:
aux_ = textFileRDD.take(2)
feature_names = aux_[0]
feature_example = aux_[1]

In [18]:
print ('Feature Names ' , feature_names)

Feature Names  ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']


In [19]:
print ('Feature Example ' , feature_example)

Feature Example  ['2008', '1', '3', '4', '2003', '1955', '2211', '2225', 'WN', '335', 'N712SW', '128', '150', '116', '-14', '8', 'IAD', 'TPA', '810', '4', '8', '0', '', '0', 'NA', 'NA', 'NA', 'NA', 'NA']


In [20]:
print ("Number of features = " , len(feature_example))

Number of features =  29


In [35]:
# ### Creating a SQL Dataframe from RDD
# 
# We now create a SQL DataFrame, this entity is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in Python, but with richer optimizations under the hood. We will utilize the recently created Spark RDD and use the Spark SQL context to create the desired data frame,

# We first create function that will allow to parse a record of our RDD into the desired format. As a reference we take a look at features_names and feature_example we just created above



def parse(r):
    try:
        x=Row(Year=int(r[0]),\
          Month=int(r[1]),\
          DayofMonth=int(r[2]),\
          DayOfWeek=int(r[3]),\
          DepTime=int(float(r[4])), \
          CRSDepTime=int(r[5]),\
          ArrTime=int(float(r[6])),\
          CRSArrTime=int(r[7]), \
          UniqueCarrier=r[8],\
          DepDelay=int(float(r[15])),\
          Origin=r[16],\
          Dest=r[17], \
          Distance=int(float(r[18])),\
          CarrierDelay=int(float(r[24])),\
          WeatherDelay=int(float(r[25])),\
          NASDelay= int(float(r[26])),\
          SecurityDelay=int(float(r[27])),\
          LateAircraftDelay=int(float(r[28])))  
    except:
        x=None  
    return x

rowRDD = textRDD.map(lambda r: parse(r)).filter(lambda r:r != None)
airline_df = sqlContext.createDataFrame(rowRDD)

In [36]:
# We add a new column to our data frame, **DepDelayed**, a binary variable:
# - **True**, for flights that have > 15 minutes of delay
# - **False**, for flights that have <= 15 minutes of delay
# 
# We will later use **Depdelayed** as the target/label column in the classification process.


airline_df = airline_df.withColumn('DepDelayed', airline_df['DepDelay']>15)

In [37]:
airline_df.take(5)

[Row(ArrTime=1959, CRSArrTime=1925, CRSDepTime=1755, CarrierDelay=2, DayOfWeek=4, DayofMonth=3, DepDelay=34, DepTime=1829, Dest='BWI', Distance=515, LateAircraftDelay=32, Month=1, NASDelay=0, Origin='IND', SecurityDelay=0, UniqueCarrier='WN', WeatherDelay=0, Year=2008, DepDelayed=True),
 Row(ArrTime=2037, CRSArrTime=1940, CRSDepTime=1830, CarrierDelay=10, DayOfWeek=4, DayofMonth=3, DepDelay=67, DepTime=1937, Dest='LAS', Distance=1591, LateAircraftDelay=47, Month=1, NASDelay=0, Origin='IND', SecurityDelay=0, UniqueCarrier='WN', WeatherDelay=0, Year=2008, DepDelayed=True),
 Row(ArrTime=1845, CRSArrTime=1725, CRSDepTime=1510, CarrierDelay=8, DayOfWeek=4, DayofMonth=3, DepDelay=94, DepTime=1644, Dest='MCO', Distance=828, LateAircraftDelay=72, Month=1, NASDelay=0, Origin='IND', SecurityDelay=0, UniqueCarrier='WN', WeatherDelay=0, Year=2008, DepDelayed=True),
 Row(ArrTime=1640, CRSArrTime=1625, CRSDepTime=1425, CarrierDelay=3, DayOfWeek=4, DayofMonth=3, DepDelay=27, DepTime=1452, Dest='PHX',

In [38]:
airline_df.take(10)

[Row(ArrTime=1959, CRSArrTime=1925, CRSDepTime=1755, CarrierDelay=2, DayOfWeek=4, DayofMonth=3, DepDelay=34, DepTime=1829, Dest='BWI', Distance=515, LateAircraftDelay=32, Month=1, NASDelay=0, Origin='IND', SecurityDelay=0, UniqueCarrier='WN', WeatherDelay=0, Year=2008, DepDelayed=True),
 Row(ArrTime=2037, CRSArrTime=1940, CRSDepTime=1830, CarrierDelay=10, DayOfWeek=4, DayofMonth=3, DepDelay=67, DepTime=1937, Dest='LAS', Distance=1591, LateAircraftDelay=47, Month=1, NASDelay=0, Origin='IND', SecurityDelay=0, UniqueCarrier='WN', WeatherDelay=0, Year=2008, DepDelayed=True),
 Row(ArrTime=1845, CRSArrTime=1725, CRSDepTime=1510, CarrierDelay=8, DayOfWeek=4, DayofMonth=3, DepDelay=94, DepTime=1644, Dest='MCO', Distance=828, LateAircraftDelay=72, Month=1, NASDelay=0, Origin='IND', SecurityDelay=0, UniqueCarrier='WN', WeatherDelay=0, Year=2008, DepDelayed=True),
 Row(ArrTime=1640, CRSArrTime=1625, CRSDepTime=1425, CarrierDelay=3, DayOfWeek=4, DayofMonth=3, DepDelay=27, DepTime=1452, Dest='PHX',

In [39]:
# define hour function to obtain hour of day
def hour_ex(x): 
    h = int(str(int(x)).zfill(4)[:2])
    return h

# register as a UDF 
f = udf(hour_ex, IntegerType())

#CRSDepTime: scheduled departure time (local, hhmm)
airline_df = airline_df.withColumn('hour', f(airline_df.CRSDepTime))
airline_df.registerTempTable("airlineDF")

In [40]:
#Exploration: What are the primary causes for flight delays
cause_delay = sqlContext.sql("SELECT sum(WeatherDelay) Weather,sum(NASDelay) NAS,sum(SecurityDelay) Security,sum(LateAircraftDelay) lateAircraft,sum(CarrierDelay) Carrier FROM airlineDF ")
df_cause_delay = cause_delay.toPandas()
df_cause_delay.head()

,Weather,NAS,Security,lateAircraft,Carrier
0,4633717,26171501,114316,31670242,24048217


In [41]:
#Exploration: Which Airports have the Most Delays
groupedDelay = sqlContext.sql("SELECT Origin, count(*) conFlight,avg(DepDelay) delay \
                                FROM airlineDF \
                                GROUP BY Origin")
df_origin = groupedDelay.toPandas()
df_origin.head()


,Origin,conFlight,delay
0,BGM,115,62.982609
1,PSE,81,54.493827
2,DLG,55,35.927273
3,INL,1,23.000000
4,MSY,7693,48.385155


In [43]:
df_origin.sort_values('delay',ascending=0).head()

,Origin,conFlight,delay
246,ACY,14,144.142857
164,CMX,35,110.342857
135,PIR,1,99.000000
250,PLN,22,86.909091
169,SPI,338,85.928994


In [46]:
! ls -lrta

total 2022912
-rw-------  1 s7ed-a18f3badb92bc2-a9f6794a31ec users 689413344 Dec  9  2014 2008.csv
drwx------  2 s7ed-a18f3badb92bc2-a9f6794a31ec users      4096 May 26 02:48 MNIST_data
drwx------ 11 s7ed-a18f3badb92bc2-a9f6794a31ec users      4096 Jul 20 06:55 ..
drwx------  3 s7ed-a18f3badb92bc2-a9f6794a31ec users      4096 Jul 20 07:22 .
-rw-------  1 s7ed-a18f3badb92bc2-a9f6794a31ec users   1068028 Jul 20 07:22 airports.dat


In [94]:
#o map each Airport to corresponding Long and Lat,load the dataset needed


#df = pd.read_csv('airports.dat', index_col=0, names = ['name', 'city', 'country','IATA','ICAO','lat','lng','alt','TZone','DST','Tz'], header=0)
#df = pd.read_csv('airports.dat', header=0, names = ['name', 'city', 'country','IATA','ICAO','lat','lng','alt','TZone','DST','Tz'])
df= pd.read_csv('airports.dat', header=0, names = ['name', 'city', 'country','IATA','ICAO','lat','lng','alt','TZone','DST','Tz'])
#df = pd.read_csv('airports.dat', index_col=0,names = ['name', 'city', 'country','IATA','ICAO','lat','lng','alt','TZone','DST','Tz'], header=0)
df.head()

,,,name,city,country,IATA,ICAO,lat,lng,alt,TZone,DST,Tz
2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
6,Wewak International Airport,Wewak,Papua New Guinea,WWK,AYWK,-3.583830,143.669006,19,10,U,Pacific/Port_Moresby,airport,OurAirports


In [95]:
df_airports = pd.merge(df_origin, df, left_on = 'Origin', right_on = 'IATA')
df_airports.head()

,Origin,conFlight,delay,name,city,country,IATA,ICAO,lat,lng,alt,TZone,DST,Tz


In [97]:
df_airports.sort_values('delay',ascending=0).head(10)

,Origin,conFlight,delay,name,city,country,IATA,ICAO,lat,lng,alt,TZone,DST,Tz


In [98]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def zscore(x):
    return (x-np.average(x))/np.std(x)

In [ ]:
! conda install -c conda-forge basemap-data-hires=1.0.8.dev0 -y

In [100]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

rcParams['figure.figsize'] = (14,10)


my_map = Basemap(projection='merc',
            resolution = 'l', area_thresh = 1000.0,
            llcrnrlon=-130, llcrnrlat=22, #min longitude (llcrnrlon) and latitude (llcrnrlat)
            urcrnrlon=-60, urcrnrlat=50) #max longitude (urcrnrlon) and latitude (urcrnrlat)

my_map.drawcoastlines()
my_map.drawcountries()
my_map.drawmapboundary()
my_map.fillcontinents(color = 'white', alpha = 0.3)
my_map.shadedrelief()

# To create a color map
colors = plt.get_cmap('hot')(np.linspace(0.0, 1.0, 30))
colors=np.flipud(colors)

#----- Scatter -------
countrange=max(df_airports['conFlight'])-min(df_airports['conFlight'])
al=np.array([sigmoid(x) for x in zscore(df_airports['delay'])])
xs,ys = my_map(np.asarray(df_airports['lng']), np.asarray(df_airports['lat']))
val=df_airports['conFlight']*4000.0/countrange

my_map.scatter(xs, ys,  marker='o', s= val, alpha = 0.8,color=colors[(al*20).astype(int)])

#----- Text -------
df_text=df_airports[(df_airports['conFlight']>60000) & (df_airports['IATA'] != 'HNL')]
xt,yt = my_map(np.asarray(df_text['lng']), np.asarray(df_text['lat']))
txt=np.asarray(df_text['IATA'])
zp=zip(xt,yt,txt)
for row in zp:
    #print zp[2]
    plt.text(row[0],row[1],row[2], fontsize=10, color='blue',)

print("Each marker is an airport.")
print("Size of markers: Airport Traffic (larger means higher number of flights in year)")
print("Color of markers: Average Flight Delay (Redder means longer delays)")

plt.show()

ImportError: No module named 'mpl_toolkits.basemap'

In [ ]:

#Exploration: Route delay
#Which Routes are typically the most delayed?
grp_rout_Delay = sqlContext.sql("SELECT Origin, Dest, count(*) traffic,avg(Distance) avgDist,\
                                    avg(DepDelay) avgDelay\
                                FROM airlineDF \
                                GROUP BY Origin,Dest")
rout_Delay = grp_rout_Delay.toPandas()

In [ ]:
df_airport_rout1 = pd.merge(rout_Delay, df, left_on = 'Origin', right_on = 'IATA')
df_airport_rout2 = pd.merge(df_airport_rout1, df, left_on = 'Dest', right_on = 'IATA')
df_airport_rout = df_airport_rout2[["Origin","lat_x","lng_x","Dest","lat_y","lng_y",\
                                    "avgDelay", "traffic"]]

In [ ]:
df_airport_rout.sort('avgDelay',ascending=0).head()

In [ ]:
rcParams['figure.figsize'] = (14,10)


my_map = Basemap(projection='merc',
            resolution = 'l', area_thresh = 1000.0,
            llcrnrlon=-130, llcrnrlat=22, #min longitude (llcrnrlon) and latitude (llcrnrlat)
            urcrnrlon=-60, urcrnrlat=50) #max longitude (urcrnrlon) and latitude (urcrnrlat)

my_map.drawcoastlines()
my_map.drawcountries()
my_map.drawmapboundary()
my_map.fillcontinents(color = 'white', alpha = 0.3)
my_map.shadedrelief()

delay=np.array([sigmoid(x) for x in zscore(df_airports["delay"])])
colors = plt.get_cmap('hot')(np.linspace(0.0, 1.0, 40))
colors=np.flipud(colors)
xs,ys = my_map(np.asarray(df_airports['lng']), np.asarray(df_airports['lat']))
xo,yo = my_map(np.asarray(df_airport_rout['lng_x']), np.asarray(df_airport_rout['lat_x']))
xd,yd = my_map(np.asarray(df_airport_rout['lng_y']), np.asarray(df_airport_rout['lat_y']))

my_map.scatter(xs, ys,  marker='o',  alpha = 0.8,color=colors[(delay*20).astype(int)])


al=np.array([sigmoid(x) for x in zscore(df_airport_rout["avgDelay"])])
f=zip(xo,yo,xd,yd,df_airport_rout['avgDelay'],al)
for row in f:
    plt.plot([row[0],row[2]], [row[1],row[3]],'-',alpha=0.07, \
             color=colors[(row[5]*30).astype(int)] )
    

for row in zp:
    plt.text(row[0],row[1],row[2], fontsize=10, color='blue',)

print("Each line represents a route from the Origin to Destination airport.")
print("The redder line, the higher probablity of delay.")
    
plt.show()

In [ ]:
#Exploration: Airport Origin delay per month
Origin_Airport="SJC"

In [ ]:
df_ORG = sqlContext.sql("SELECT * from airlineDF WHERE origin='"+ Origin_Airport+"'")
df_ORG.registerTempTable("df_ORG")
df_ORG.select('ArrTime','CRSArrTime','CRSDepTime',\
              'DayOfWeek','DayofMonth','DepDelay','DepTime','Dest').show(2)

In [ ]:
print ("total flights from this ariport: " + str(df_ORG.count()))

In [ ]:
grp_carr = sqlContext.sql("SELECT  UniqueCarrier,month, avg(DepDelay) avgDelay from df_ORG \
                            WHERE DepDelayed=True \
                            GROUP BY UniqueCarrier,month")
s = grp_carr.toPandas()

In [ ]:
ps = s.pivot(index='month', columns='UniqueCarrier', values='avgDelay')[['AA','UA','US']]

In [ ]:
rcParams['figure.figsize'] = (8,5)
ps.plot(kind='bar', colormap='Greens');
plt.xlabel('Average delay')
plt.ylabel('Month')
plt.title('How much delay does each carrier has in each month?')

In [ ]:

#We see that average delay in this year is is highest in November and October in this airport.
#Exploration: Airport Origin delay per day/hour
hour_grouped = df_ORG.filter(df_ORG['DepDelayed']).select('DayOfWeek','hour','DepDelay').groupby('DayOfWeek','hour').mean('DepDelay')

In [ ]:
#Modeling: Logistic Regression
df_model=df_ORG
stringIndexer1 = StringIndexer(inputCol="Origin", outputCol="originIndex")
model_stringIndexer = stringIndexer1.fit(df_model)
indexedOrigin = model_stringIndexer.transform(df_model)
encoder1 = OneHotEncoder(dropLast=False, inputCol="originIndex", outputCol="originVec")
df_model = encoder1.transform(indexedOrigin)

In [ ]:
assembler = VectorAssembler(
    inputCols = ['Year','Month','DayofMonth','DayOfWeek','hour','Distance','originVec'],
    outputCol = "features")
output = assembler.transform(df_model)
airlineRDD=output.map(lambda row: LabeledPoint([0,1][row['DepDelayed']],row['features']))

In [ ]:
#  Spliting dataset into train and test dtasets
trainRDD,testRDD=airlineRDD.randomSplit([0.7,0.3])

In [ ]:
# Build the model
model = LogisticRegressionWithLBFGS.train(trainRDD)

In [ ]:
#Model Evaluation
# Evaluating the model on testing data
labelsAndPreds = testRDD.map(lambda p: (p.label, model.predict(p.features)))

In [ ]:
def conf(r):
    if r[0] == r[1] ==1: x= 'TP'
    if r[0] == r[1] ==0: x= 'TN'
    if r[0] == 1 and  r[1] ==0: x= 'FN'
    if r[0] == 0 and  r[1] ==1: x= 'FP'
    return (x)
acc1 = labelsAndPreds.map(lambda (v, p): ((v, p),1)).reduceByKey(lambda a, b: a + b).take(5)
acc = [(conf(x[0]),x[1]) for x in acc1]

In [ ]:
TP=TN=FP=FN=0.0
for x in acc: 
    if x[0]=='TP': TP= x[1]
    if x[0]=='TN': TN= x[1]
    if x[0]=='FP': FP= x[1]
    if x[0]=='FN': FN= x[1]
eps = sys.float_info.epsilon
Accuracy = (TP+TN) / (TP + TN+ FP+FN+eps) 
print ("Model Accuracy for JFK: %1.2f %%" % (Accuracy*100))